In [ ]:
#access here for colab with the code which is already runned:https://colab.research.google.com/drive/1-4uDO4KOJHpKUAD3d21HS3iJGw1B7VXn#scrollTo=NsVi7wyH0wRK


#Data Preprocessing
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset (replace with your custom dataset if needed)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize the data to [0, 1] range
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels (10 classes)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Function to plot sample images
def plot_images(images, labels, class_names):
    plt.figure(figsize=(10, 10))
    batch_size = len(images)  # Dynamically adjust based on batch size
    rows = int(np.sqrt(batch_size))
    cols = int(np.ceil(batch_size / rows))

    for i in range(batch_size):
        plt.subplot(rows, cols, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i])
        plt.xlabel(class_names[np.argmax(labels[i])])
    plt.show()

# Display a few sample images
plot_images(X_train[:9], y_train[:9], class_names)

In [ ]:
#Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

# Fit the data generator on the training data
datagen.fit(X_train)

# Display augmented images
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    plot_images(X_batch, y_batch, class_names)
    break

In [ ]:
# Import necessary libraries
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# Define the model architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

In [ ]:
# Define the Adam optimizer
optimizer = Adam(learning_rate=0.0005)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

# No augmentation for validation data
test_datagen = ImageDataGenerator()

# Fit the data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=64)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=64)

# Define callbacks
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  # Filepath where the model is saved
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # Save the model with the minimum validation loss
    save_best_only=True,  # Only save the model when validation loss improves
    verbose=1  # Print information when the model is saved
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,  # Training data generator
    epochs=150,  # Number of epochs to train for
    validation_data=validation_generator,  # Validation data generator
    callbacks=[checkpoint, early_stopping, reduce_lr],  # Use ModelCheckpoint, EarlyStopping, and ReduceLROnPlateau callbacks
    verbose=2  # Display detailed logs for each epoch
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
#visualising metrics
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Bar chart for final training and validation accuracy
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
plt.figure(figsize=(6, 4))
plt.bar(['Training Accuracy', 'Validation Accuracy'], [train_acc, val_acc], color=['blue', 'green'])
plt.title('Final Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.show()

# Bar chart for final training and validation loss
train_loss = history.history['loss'][-1]  # Last training loss value
val_loss = history.history['val_loss'][-1]  # Last validation loss value

# Create the bar chart
plt.figure(figsize=(6, 4))
plt.bar(['Training Loss', 'Validation Loss'], [train_loss, val_loss], color=['blue', 'green'])
plt.title('Final Training and Validation Loss')
plt.ylabel('Loss')
plt.grid(True)  # Add grid lines for better readability
plt.show()

In [ ]:
#model evaluation +visualisation

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predict on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true = np.argmax(y_test, axis=1)  # Convert true labels to class labels

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Classification report
report = classification_report(y_true, y_pred_classes, target_names=class_names, output_dict=True)
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# Extracting precision, recall, and F1-score for bar chart plotting
precision = [report[label]['precision'] for label in class_names]
recall = [report[label]['recall'] for label in class_names]
f1_score = [report[label]['f1-score'] for label in class_names]

# Plotting Precision, Recall, and F1-score bar charts
x = np.arange(len(class_names))  # Label locations
width = 0.2  # Width of the bars

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width, precision, width, label='Precision', color='skyblue')
rects2 = ax.bar(x, recall, width, label='Recall', color='lightgreen')
rects3 = ax.bar(x + width, f1_score, width, label='F1-Score', color='salmon')

# Add some text for labels, title, and custom x-axis tick labels
ax.set_xlabel('Class')
ax.set_ylabel('Score')
ax.set_title('Precision, Recall, and F1-Score per Class')
ax.set_xticks(x)
ax.set_xticklabels(class_names)
ax.legend()

# Display the bar chart
plt.show()

# Plotting total predictions per class
total_predictions = np.sum(conf_matrix, axis=1)
plt.figure(figsize=(10, 6))
sns.barplot(x=class_names, y=total_predictions, palette='Blues_d', hue=class_names, legend=False)
plt.title('Total Predictions per Class')
plt.xlabel('Class')
plt.ylabel('Number of Predictions')
plt.show()


In [ ]:
# Import VGG16 from Keras with pre-trained ImageNet weights
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


# Load CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


(X_train, y_train), (X_test, y_test) = cifar10.load_data()


# Normalize the data to [0, 1] range
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)


# Load the VGG16 model with ImageNet weights, exclude the top (fully-connected) layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


# Freeze all layers in the VGG16 base model initially
vgg_base.trainable = False


# Add custom top layers for CIFAR-10 classification
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)


# Create a new model combining the VGG16 base and the custom layers
model_vgg = Model(inputs=vgg_base.input, outputs=output)


# Compile the model
model_vgg.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])


# Define callbacks for training
checkpoint = ModelCheckpoint(
    filepath='vgg16_finetuned_best_model.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)


# Train the model with the frozen VGG16 layers
history_vgg = model_vgg.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=2
)

# Unfreeze the last 10 layers of VGG16 for fine-tuning
for layer in vgg_base.layers[-10:]:
    layer.trainable = True


# Recompile the model after unfreezing 10 layers
model_vgg.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# Fine-tune the model again
history_fine_tune = model_vgg.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=40,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=2
)


# Unfreeze more layers (up to 20) for further fine-tuning
for layer in vgg_base.layers[-20:]:
    layer.trainable = True


# Recompile the model again after unfreezing more layers
model_vgg.compile(optimizer=Adam(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])


# Fine-tune with 20 unfrozen layers
history_fine_tune_more = model_vgg.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=2
)


# Evaluate the model on the test set
test_loss, test_acc = model_vgg.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc * 100:.2f}%")


# If test accuracy is above 95%, save the final model
if test_acc >= 0.95:
    model_vgg.save('final_vgg16_model_95_acc.keras')


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


# Visualizing metrics and comparing with custom CNN
plt.figure(figsize=(12, 5))

# Plot training and validation accuracy for VGG16
plt.plot(history_vgg.history['accuracy'], label='VGG16 Training Accuracy', linestyle='--', color='blue')
plt.plot(history_vgg.history['val_accuracy'], label='VGG16 Validation Accuracy', linestyle='-', color='blue')
plt.plot(history_fine_tune.history['accuracy'], label='VGG16 Fine-tune Training Accuracy', linestyle='--', color='green')
plt.plot(history_fine_tune.history['val_accuracy'], label='VGG16 Fine-tune Validation Accuracy', linestyle='-', color='green')
plt.plot(history_fine_tune_more.history['accuracy'], label='VGG16 Fine-tune More Training Accuracy', linestyle='--', color='red')
plt.plot(history_fine_tune_more.history['val_accuracy'], label='VGG16 Fine-tune More Validation Accuracy', linestyle='-', color='red')

plt.title('Training and Validation Accuracy for VGG16')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training and validation loss for VGG16
plt.figure(figsize=(12, 5))
plt.plot(history_vgg.history['loss'], label='VGG16 Training Loss', linestyle='--', color='blue')
plt.plot(history_vgg.history['val_loss'], label='VGG16 Validation Loss', linestyle='-', color='blue')
plt.plot(history_fine_tune.history['loss'], label='VGG16 Fine-tune Training Loss', linestyle='--', color='green')
plt.plot(history_fine_tune.history['val_loss'], label='VGG16 Fine-tune Validation Loss', linestyle='-', color='green')
plt.plot(history_fine_tune_more.history['loss'], label='VGG16 Fine-tune More Training Loss', linestyle='--', color='red')
plt.plot(history_fine_tune_more.history['val_loss'], label='VGG16 Fine-tune More Validation Loss', linestyle='-', color='red')

plt.title('Training and Validation Loss for VGG16')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Confusion Matrix and Classification Report for VGG16
y_pred_vgg = model_vgg.predict(X_test)
y_pred_classes_vgg = np.argmax(y_pred_vgg, axis=1)
y_true_vgg = np.argmax(y_test, axis=1)

conf_matrix_vgg = confusion_matrix(y_true_vgg, y_pred_classes_vgg)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix_vgg, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(10), yticklabels=np.arange(10))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for VGG16 Model')
plt.show()
report_vgg = classification_report(y_true_vgg, y_pred_classes_vgg, output_dict=True)
print(classification_report(y_true_vgg, y_pred_classes_vgg))

# Extract precision, recall, and F1-score for bar chart plotting for VGG16
precision_vgg = [report_vgg[str(i)]['precision'] for i in range(10)]
recall_vgg = [report_vgg[str(i)]['recall'] for i in range(10)]
f1_score_vgg = [report_vgg[str(i)]['f1-score'] for i in range(10)]

# Plotting Precision, Recall, and F1-score bar charts
x = np.arange(10)  # Label locations
width = 0.2  # Width of the bars

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width, precision_vgg, width, label='Precision', color='skyblue')
rects2 = ax.bar(x, recall_vgg, width, label='Recall', color='lightgreen')
rects3 = ax.bar(x + width, f1_score_vgg, width, label='F1-Score', color='salmon')

# Add some text for labels, title, and custom x-axis tick labels
ax.set_xlabel('Class')
ax.set_ylabel('Score')
ax.set_title('Precision, Recall, and F1-Score per Class for VGG16')
ax.set_xticks(x)
ax.set_xticklabels(range(10))
ax.legend()

# Display the bar chart
plt.show()